In [ ]:
!nvidia-smi

Wed Aug 24 11:33:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import cv2
import numpy as np
 
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.utils import np_utils
from keras.datasets import mnist
 
epochs = 10
batch_size = 50
row_col = 48
 
# 原始的 MNIST 是 6000 筆 28*28 灰階
def load_data():
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train, y_train = X_train[:5000], y_train[:5000]
  X_test, y_test = X_test[5000:6000], y_test[5000:6000]
  X_train = [cv2.cvtColor(cv2.resize(i, (row_col, row_col)), cv2.COLOR_GRAY2RGB)
            for i in X_train]
  X_train = np.concatenate([arr[np.newaxis] for arr in X_train]).astype('float32')
  X_test = [cv2.cvtColor(cv2.resize(i, (row_col, row_col)), cv2.COLOR_GRAY2RGB)
            for i in X_test]
  X_test = np.concatenate([arr[np.newaxis] for arr in X_test]).astype('float32')
 
  X_train = X_train / 255
  X_test = X_test / 255
 
  y_train_ohe = np_utils.to_categorical(y_train, 10)
  y_test_ohe = np_utils.to_categorical(y_test, 10)
 
  return (X_train, y_train_ohe), (X_test, y_test_ohe)
 
 
def load_model():
 
  base_network = VGG16(include_top=False, weights='imagenet', input_shape=(row_col, row_col, 3))
 
  # 凍結預設的參數
  for layer in base_network.layers:
      layer.trainable = False
 
  # 接上自行定義的全連結層
  model = Flatten()(base_network.output)
  model = Dense(4096, activation='relu', name='full_connect_1')(model)
  model = Dense(4096, activation='relu', name='full_connect_2')(model)
  model = Dropout(0.5)(model)
  model = Dense(10, activation='softmax', name='prediction')(model)
  model = Model(base_network.input, model, name='my_model')
 
  return model
  
  
# 載入模型
model = load_model()
model.summary()
 
# 載入訓練資料
(x_train, y_train_ohe), (x_test, y_test_ohe) = load_data()
print('Train Size:', x_train.shape)
print('Test Size:', x_test.shape)
 
#定義 loss function
from tensorflow.keras.optimizers import SGD
sgd = SGD(lr=0.05, decay=1e-5)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
 
# 開始訓練
model.fit(x_train, y_train_ohe, validation_data=(x_test, y_test_ohe), epochs=epochs, batch_size=batch_size)
 
# 輸出結果
print('Train Acc:', model.evaluate(x_train, y_train_ohe)[1])
print('Test Acc:', model.evaluate(x_test, y_test_ohe)[1])

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
100/100 [==============================] - 14s 27ms/step - loss: 1.3704 - accuracy: 0.5574 - val_loss: 0.7938 - val_accuracy: 0.7630
Epoch 2/10
100/100 [==============================] - 2s 25ms/step - loss: 0.7356 - accuracy: 0.7682 - val_loss: 0.6517 - val_accuracy: 0.7720
Epoch 3/10
100/100 [==============================] - 2s 25ms/step - loss: 0.5513 - accuracy: 0.8264 - val_loss: 0.5483 - val_accuracy: 0.7780
Epoch 4/10
100/100 [==============================] - 3s 25ms/step - loss: 0.4619 - accuracy: 0.8488 - val_loss: 0.3378 - val_accuracy: 0.8900
Epoch 5/10
100/100 [==============================] - 3s 28ms/step - loss: 0.4277 - accuracy: 0.8642 - val_loss: 0.3214 - val_accuracy: 0.8940
Epoch 6/10
100/100 [==============================] - 3s 28ms/step - loss: 0.4153 - accuracy: 0.8730 - val_loss: 0.3412 - val_accuracy: 0.8730
Epoch 7/10
100/100 [==============================] - 3s 28ms/step - loss: 0.3296 - accuracy: 0.8930 - val_loss: 0.2764 - val_accuracy: 0.915

In [7]:
import tensorflow as tf
tf.keras.preprocessing.text.text_to_word_sequence("Hello World")

['hello', 'world']

In [10]:
x = [1,2,3,4]
y = [5,6,7,8]
print(list(zip(x, y)))

[([1, 2, 3, 4], 5)]
